# RAGatouille + ColBERT로 고급 검색 구현하기

이 노트북에서는 **RAGatouille**과 **ColBERT** 모델을 사용하여 고급 검색 시스템을 구축하는 방법을 알아봅니다.

## ColBERT란?

**ColBERT**(Contextualized Late Interaction over BERT)는 효율적인 문서 검색을 위한 신경망 모델입니다.

### 일반 임베딩 vs ColBERT

| 방식 | 임베딩 | 검색 방식 |
|------|--------|----------|
| **일반** | 문서 → 단일 벡터 | 벡터 유사도 |
| **ColBERT** | 문서 → 토큰별 벡터 | Late Interaction |

### ColBERT의 장점

1. **높은 정확도**: 토큰 수준 매칭으로 세밀한 검색
2. **효율성**: 사전 계산된 임베딩으로 빠른 검색
3. **설명 가능성**: 어떤 토큰이 매칭되었는지 확인 가능

## RAGatouille이란?

ColBERT를 쉽게 사용할 수 있게 해주는 Python 라이브러리입니다.

```python
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
```

## 주의사항

- **Windows 미지원**
- Python 전용
- GPU 권장 (CPU도 가능하지만 느림)

---

# 1. 패키지 설치

In [ ]:
!pip install -q ragatouille transformers requests

# 2. ColBERT 모델 로드

In [ ]:
from ragatouille import RAGPretrainedModel

# ColBERT v2 모델 로드
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

print("✅ ColBERT 모델 로드 완료")

# 3. 위키백과에서 문서 가져오기

In [ ]:
import requests

def get_wikipedia_page(title: str):
    """
    위키백과의 페이지를 불러온다.
    
    :param title: 위키백과 페이지의 제목
    :return: 페이지의 전체 텍스트
    """
    URL = "https://en.wikipedia.org/w/api.php"
    
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }
    
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1"}
    
    response = requests.get(URL, params=params, headers=headers)
    data = response.json()
    
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"] if "extract" in page else None

# 미야자키 하야오 위키백과 문서 가져오기
full_document = get_wikipedia_page("Hayao_Miyazaki")

print(f"문서 길이: {len(full_document)}자")
print(f"\n미리보기:\n{full_document[:500]}...")

# 4. 인덱스 생성

**코드 설명:**

```python
RAG.index(
    collection=[document],     # 인덱싱할 문서들
    index_name="index_name",   # 인덱스 이름
    max_document_length=180,   # 청크 최대 길이
    split_documents=True,      # 자동 분할 여부
)
```

In [ ]:
# ColBERT 인덱스 생성
RAG.index(
    collection=[full_document],
    index_name="Miyazaki-index",
    max_document_length=180,
    split_documents=True,
)

print("✅ 인덱스 생성 완료")

# 5. ColBERT로 검색

In [ ]:
# 검색 수행
query = "What animation studio did Miyazaki found?"
results = RAG.search(query=query, k=3)

print(f"쿼리: '{query}'\n")
print("=== 검색 결과 ===")
for i, result in enumerate(results):
    print(f"\n[{i+1}] Score: {result['score']:.4f}")
    print(f"    {result['content'][:200]}...")

# 6. LangChain Retriever로 변환

In [ ]:
# LangChain Retriever로 변환
retriever = RAG.as_langchain_retriever(k=3)

# Retriever 사용
docs = retriever.invoke("What animation studio did Miyazaki found?")

print("=== LangChain Retriever 결과 ===")
for i, doc in enumerate(docs):
    print(f"\n[{i+1}] {doc.page_content[:150]}...")

# 7. RAG 체인에서 사용

In [ ]:
# Ollama 설치 (RAG 체인용)
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh
subprocess.Popen(['ollama', 'serve'])
time.sleep(3)
!ollama pull llama3.2
!pip install -q langchain-ollama

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RAG 체인 구성
llm = ChatOllama(model='llama3.2')

prompt = ChatPromptTemplate.from_template('''Answer the question based on the context:

Context: {context}

Question: {question}

Answer:''')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# RAG 체인 실행
answer = rag_chain.invoke("What animation studio did Miyazaki found?")

print("=== RAG 응답 ===")
print(answer)

---

## RAGatouille 주요 메서드

| 메서드 | 설명 |
|--------|------|
| `from_pretrained()` | 사전훈련 모델 로드 |
| `index()` | 문서 인덱싱 |
| `search()` | 검색 수행 |
| `as_langchain_retriever()` | LangChain Retriever 변환 |

## 일반 임베딩 vs ColBERT 비교

| 특성 | 일반 임베딩 | ColBERT |
|------|------------|--------|
| 임베딩 차원 | 문서당 1개 벡터 | 토큰당 1개 벡터 |
| 검색 정확도 | 보통 | 높음 |
| 인덱스 크기 | 작음 | 큼 |
| 검색 속도 | 빠름 | 약간 느림 |

## 사용 시나리오

- **높은 정확도 필요**: 법률, 의료 문서 검색
- **세밀한 매칭 필요**: 기술 문서, 코드 검색
- **설명 가능성 필요**: 검색 결과 근거 제시